In [3]:
from csv import reader
from collections import defaultdict
from itertools import chain, combinations
from optparse import OptionParser

In [11]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))


def getFromFile(fname):
    itemSets = []
    itemSet = set()

    with open(fname, 'r') as file:
        csv_reader = reader(file)
        for line in csv_reader:
            line = list(filter(None, line))
            record = set(line)
            for item in record:
                itemSet.add(frozenset([item]))
            itemSets.append(record)
    return itemSet, itemSets


def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList))
        if(support >= minSup):
            freqItemSet.add(item)

    return freqItemSet


def getUnion(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet


def associationRule(freqItemSet, itemSetWithSup, minConf):
    rules = []
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            for s in subsets:
                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules


def getItemSetFromList(itemSetList):
    tempItemSet = set()

    for itemSet in itemSetList:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))

    return tempItemSet

In [5]:
def apriori(itemSetList, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    # Final result global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules

def aprioriFromFile(fname, minSup, minConf):
    C1ItemSet, itemSetList = getFromFile(fname)

    # Final result global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules

In [8]:
records = []
with open('records.txt', 'r') as f:
    for line in f:
        records.append(set(line.strip().split()))
records[:5]

[{'ELE17451', 'ELE89019', 'FRO11987', 'GRO99222', 'SNA90258'},
 {'ELE17451',
  'ELE26917',
  'ELE52966',
  'ELE91550',
  'FRO12685',
  'FRO84225',
  'FRO90334',
  'GRO12298',
  'GRO99222',
  'SNA11465',
  'SNA30755',
  'SNA80192'},
 {'DAI22896', 'ELE17451', 'FRO86643', 'GRO73461', 'SNA99873'},
 {'ELE17451', 'ELE23393', 'ELE37798', 'FRO86643', 'GRO56989', 'SNA11465'},
 {'DAI54444',
  'ELE11375',
  'ELE17451',
  'ELE28573',
  'FRO78087',
  'FRO86643',
  'GRO39357',
  'SNA11465',
  'SNA69641'}]

In [10]:
# getItemSetFromList(records)
getFromFile('tmp.csv')

({frozenset({'112'}),
  frozenset({'1034'}),
  frozenset({'1194'}),
  frozenset({'704'}),
  frozenset({'656'}),
  frozenset({'1125'}),
  frozenset({'1094'}),
  frozenset({'948'}),
  frozenset({'929'}),
  frozenset({'1289'}),
  frozenset({'1017'}),
  frozenset({'1651'}),
  frozenset({'870'}),
  frozenset({'1040'}),
  frozenset({'1250'}),
  frozenset({'1273'}),
  frozenset({'1214'}),
  frozenset({'1544'}),
  frozenset({'1516'}),
  frozenset({'783'}),
  frozenset({'647'}),
  frozenset({'1940'}),
  frozenset({'11'}),
  frozenset({'1605'}),
  frozenset({'8585'}),
  frozenset({'1182'}),
  frozenset({'1318'}),
  frozenset({'1429'}),
  frozenset({'89'})},
 [{'1125', '1289', '1544', '1651', '8585', '948'},
  {'1017', '1273', '1318', '1429', '1516', '656', '783'},
  {'1034', '112'},
  {'1040', '1094', '1182', '1194', '1214', '1250', '647', '704', '929'},
  {'1094', '11', '1605', '1940', '870', '89'}])

In [12]:
aprioriFromFile('tmp.csv', 0.01, 0.0)


({1: {frozenset({'112'}),
   frozenset({'1034'}),
   frozenset({'1194'}),
   frozenset({'704'}),
   frozenset({'656'}),
   frozenset({'1125'}),
   frozenset({'1094'}),
   frozenset({'948'}),
   frozenset({'929'}),
   frozenset({'1289'}),
   frozenset({'1017'}),
   frozenset({'1651'}),
   frozenset({'870'}),
   frozenset({'1040'}),
   frozenset({'1250'}),
   frozenset({'1273'}),
   frozenset({'1214'}),
   frozenset({'1544'}),
   frozenset({'1516'}),
   frozenset({'783'}),
   frozenset({'647'}),
   frozenset({'1940'}),
   frozenset({'11'}),
   frozenset({'1605'}),
   frozenset({'8585'}),
   frozenset({'1182'}),
   frozenset({'1318'}),
   frozenset({'1429'}),
   frozenset({'89'})},
  2: {frozenset({'1040', '929'}),
   frozenset({'1289', '8585'}),
   frozenset({'1429', '656'}),
   frozenset({'1651', '8585'}),
   frozenset({'1273', '1516'}),
   frozenset({'1040', '1194'}),
   frozenset({'1094', '870'}),
   frozenset({'11', '89'}),
   frozenset({'1273', '656'}),
   frozenset({'1194', '929'})